In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
# Load your dataset
data = pd.read_csv('/content/drive/MyDrive/SA Capstone/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')

In [4]:


# Split the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

# Create a function to preprocess the dataset for T5
def preprocess_data(data):
    inputs = "instruction: " + data["instruction"]
    targets = data["response"]
    return inputs, targets

train_inputs, train_targets = preprocess_data(train_data)
test_inputs, test_targets = preprocess_data(test_data)

In [ ]:


# Load the pre-trained T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [ ]:
# Tokenize the data
train_encodings = tokenizer(train_inputs.tolist(), padding=True, truncation=True, return_tensors="pt")
train_labels = tokenizer(train_targets.tolist(), padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(test_inputs.tolist(), padding=True, truncation=True, return_tensors="pt")
test_labels = tokenizer(test_targets.tolist(), padding=True, truncation=True, return_tensors="pt")


In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels["input_ids"][idx])
        return item

    def __len__(self):
        return len(self.labels["input_ids"])

# Create the datasets
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)


In [ ]:
from transformers import Trainer, TrainingArguments
# !pip install transformers[torch] accelerate -U
# pip install transformers[torch]

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.02,
    save_total_limit=1,
    save_steps=10_000,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-dbef36069641>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-dbef36069641>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels["input_ids"][idx])


Epoch,Training Loss,Validation Loss
1,0.292100,0.247166
2,0.253400,0.223426
3,0.240900,0.215609


<ipython-input-6-dbef36069641>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-dbef36069641>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels["input_ids"][idx])


TrainOutput(global_step=9069, training_loss=0.2869518974910748, metrics={'train_runtime': 3981.649, 'train_samples_per_second': 18.222, 'train_steps_per_second': 2.278, 'total_flos': 709599179833344.0, 'train_loss': 0.2869518974910748, 'epoch': 3.0})

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)


<ipython-input-6-dbef36069641>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-dbef36069641>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels["input_ids"][idx])


{'eval_loss': 0.21560907363891602, 'eval_runtime': 43.5546, 'eval_samples_per_second': 61.716, 'eval_steps_per_second': 7.714, 'epoch': 3.0}


In [7]:
# def generate_response(instruction):
#     inputs = tokenizer("instruction: " + instruction, return_tensors="pt", padding=True, truncation=True)
#     outputs = model.generate(inputs["input_ids"])
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_response(instruction):
    inputs = tokenizer("instruction: " + instruction, return_tensors="pt", padding=True, truncation=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Add this line
    outputs = model.generate(inputs["input_ids"],
                             max_length=256,
                              min_length=10,
                              num_beams=5,
                              early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# instruction = "Create my profile"
# response = generate_response(instruction)
# print(response)

Thank you for expressing your interest in creating a profile! I'm here to assist


In [8]:
def chat():

    print("Chatbot: Hi! I am your virtual assistance,Feel free to ask, and I'll do my best to provide you with answers and assistance..")
    print("Type 'quit' to exit the chat\n\n")

    text = input("User: ")

    while(text != 'quit'):


        response = generate_response(text)

        print(f"Chatbot: {response}\n\n")

        text = input("User: ")

In [10]:
chat()

Chatbot: Hi! I am your virtual assistance,Feel free to ask, and I'll do my best to provide you with answers and assistance..
Type 'quit' to exit the chat


User: Hi!
Chatbot: Thank you for reaching out! I'm here to assist you in resolving this matter. To get started, could you please provide me with more details about the specific issue you're facing? This will help me guide you through the process and ensure a seamless experience.


User: I need assistance for creating a new account
Chatbot: Thank you for expressing your interest in creating a new account! I'm here to assist you every step of the way. To get started, could you please provide me with your full name, email address, and a preferred username? Once I have these details, I'll guide you through the account creation process, ensuring a smooth and hassle-free experience. Rest assured, your security and privacy are of utmost importance to us, and we will handle your information with the utmost care. Let's get started on this ex

In [ ]:
# save model
model.save_pretrained('/content/drive/MyDrive/SA Capstone/model_t5_small')
tokenizer.save_pretrained('/content/drive/MyDrive/SA Capstone/tokenizer_t5_small')

('/content/drive/MyDrive/SA Capstone/tokenizer_t5_small/tokenizer_config.json',
 '/content/drive/MyDrive/SA Capstone/tokenizer_t5_small/special_tokens_map.json',
 '/content/drive/MyDrive/SA Capstone/tokenizer_t5_small/spiece.model',
 '/content/drive/MyDrive/SA Capstone/tokenizer_t5_small/added_tokens.json')

In [6]:
# load model
model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/SA Capstone/model_t5_small')
tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/SA Capstone/tokenizer_t5_small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
